In [1]:
import os
import sys
import datetime

import matlab
import pandas as pd
import numpy as np
import csv

os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/";

sys.path.append("../matlab/myBTD2/myBTD2/for_redistribution_files_only");
import myBTD2

sys.path.append("../matlab/myBTD/myBTD/for_redistribution_files_only");
import myBTD

from myPackages import myDehankelization, myHankelization

/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [12]:
def get_dim_hankel(N):
    N = int(N)
    if N & 1:
        I = J = (N + 1)/2 
    else:
        I = N/2
        J = N/2 + 1
    return int(I), int(J)


def hankelization(recording):
    n_lead, n_rec = recording.shape
    I, J = get_dim_hankel(n_rec)
    T = np.zeros((I, J, n_lead))
    my_hankelization = myHankelization.initialize()
    for i in range(0, n_lead):
        vec = matlab.double(recording[i].tolist(), size=recording[i].shape)
        T[:, :, i] = my_hankelization.hankelization(vec)
    my_hankelization.terminate()
    T = matlab.double(T.tolist(), size=T.shape)
    return T


def run_btd(params):
    btd = myBTD.initialize()
    result, output = btd.myBTD(params['rank'], 
                               params['multirank'], 
                               params['max_iter'], 
                               params['tol_fun'], 
                               params['tol_x'], 
                               params['display'], 
                               params['recording_hankel'],
                               nargout=2)
    btd.terminate()
    return result, output


def run_btd2(params):
    btd = myBTD2.initialize()
    result, output = btd.myBTD2(params['rank'], 
                                params['lr'], 
                                params['max_iter'], 
                                params['tol_fun'], 
                                params['tol_x'], 
                                params['display'], 
                                params['recording_hankel'], 
                                nargout=2)
    btd.terminate()
    return result, output

In [13]:
MULTIRANK = matlab.double([40, 40, 1], size=(1, 3))
LR = matlab.double([17], size=(1, 1))
MAX_ITER = matlab.double([1000], size=(1, 1))
TOLERANCE_FUN = matlab.double([1.0E-6], size=(1, 1))
TOLERANCE_X = matlab.double([1.0E-6], size=(1, 1))
DISPLAY = matlab.logical([False], size=(1, 1))
path = f'../workdata/shura/'

LL1_MODE = True

if not os.path.exists(f'{path}'):
    os.makedirs(path) 

In [14]:
aioros_df = pd.read_pickle(f'../workdata/aioros/aioros_cpsc2018.pkl')
aioros_df.groupby(['diag', 'db']).size()

diag  db           
AF    WFDB_CPSC2018    1219
SR    WFDB_CPSC2018     918
dtype: int64

In [9]:
list(range(5, 8))

[5, 6, 7]

In [15]:
rows = aioros_df.shape[0]

for rank in range(5, 8):
    print(f'RANK: {rank}')
    
    if not os.path.exists(f'{path}/rank_{rank}'):
        os.makedirs(f'{path}/rank_{rank}')  
        
    rank_matlab = matlab.double([rank], size=(1, 1))
    
    for index, row in aioros_df.iterrows():
        recording = row['data']
        recording_hankel = hankelization(recording)
        print(f'{index}/{rows}')
                
        params = {
            'rank': rank_matlab,
            'lr': LR,
            'multirank': MULTIRANK,
            'max_iter': MAX_ITER,
            'tol_fun': TOLERANCE_FUN,
            'tol_x': TOLERANCE_X,
            'display': DISPLAY,
            'recording_hankel': recording_hankel
        }
        
        result, output = run_btd2(params) if LL1_MODE else run_btd(params)
        
        output['index'] = index
        output['rank'] = rank
        output['lr'] = LR[0][0]
        output['filename'] = row['filename']                       
        output['db'] = row['db']
        output['diagnostic'] = row['diag']
        output['datetime'] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

        with open(f'{path}/rank_{rank}/{row.name}.npy', 'wb') as file:
            np.save(file, result)

        file_exists = os.path.exists(f'{path}/results.csv')
        
        with open(f'{path}/results.csv', 'a') as f:
            headers = output.keys()
            writer = csv.DictWriter(f, delimiter=',', lineterminator='\n', fieldnames=headers)
            if not file_exists:
                writer.writeheader()
            writer.writerow(output)

RANK: 5
0/2137


/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


1/2137
2/2137
3/2137
4/2137
5/2137
6/2137
7/2137
8/2137
9/2137
10/2137
11/2137
12/2137
13/2137
14/2137
15/2137
16/2137
17/2137
18/2137
19/2137
20/2137
21/2137
22/2137
23/2137
24/2137
25/2137
26/2137
27/2137
28/2137
29/2137
30/2137
31/2137
32/2137
33/2137
34/2137
35/2137
36/2137
37/2137
38/2137
39/2137
40/2137
41/2137
42/2137
43/2137
44/2137
45/2137
46/2137
47/2137
48/2137
49/2137
50/2137
51/2137
52/2137
53/2137
54/2137
55/2137
56/2137
57/2137
58/2137
59/2137
60/2137
61/2137
62/2137
63/2137
64/2137
65/2137
66/2137
67/2137
68/2137
69/2137
70/2137
71/2137
72/2137
73/2137
74/2137
75/2137
76/2137
77/2137
78/2137
79/2137
80/2137
81/2137
82/2137
83/2137
84/2137
85/2137
86/2137
87/2137
88/2137
89/2137
90/2137
91/2137
92/2137
93/2137
94/2137
95/2137
96/2137
97/2137
98/2137
99/2137
100/2137
101/2137
102/2137
103/2137
104/2137
105/2137
106/2137
107/2137
108/2137
109/2137
110/2137
111/2137
112/2137
113/2137
114/2137
115/2137
116/2137
117/2137
118/2137
119/2137
120/2137
121/2137
122/2137
123/2137
1

924/2137
925/2137
926/2137
927/2137
928/2137
929/2137
930/2137
931/2137
932/2137
933/2137
934/2137
935/2137
936/2137
937/2137
938/2137
939/2137
940/2137
941/2137
942/2137
943/2137
944/2137
945/2137
946/2137
947/2137
948/2137
949/2137
950/2137
951/2137
952/2137
953/2137
954/2137
955/2137
956/2137
957/2137
958/2137
959/2137
960/2137
961/2137
962/2137
963/2137
964/2137
965/2137
966/2137
967/2137
968/2137
969/2137
970/2137
971/2137
972/2137
973/2137
974/2137
975/2137
976/2137
977/2137
978/2137
979/2137
980/2137
981/2137
982/2137
983/2137
984/2137
985/2137
986/2137
987/2137
988/2137
989/2137
990/2137
991/2137
992/2137
993/2137
994/2137
995/2137
996/2137
997/2137
998/2137
999/2137
1000/2137
1001/2137
1002/2137
1003/2137
1004/2137
1005/2137
1006/2137
1007/2137
1008/2137
1009/2137
1010/2137
1011/2137
1012/2137
1013/2137
1014/2137
1015/2137
1016/2137
1017/2137
1018/2137
1019/2137
1020/2137
1021/2137
1022/2137
1023/2137
1024/2137
1025/2137
1026/2137
1027/2137
1028/2137
1029/2137
1030/2137
1031/2

1751/2137
1752/2137
1753/2137
1754/2137
1755/2137
1756/2137
1757/2137
1758/2137
1759/2137
1760/2137
1761/2137
1762/2137
1763/2137
1764/2137
1765/2137
1766/2137
1767/2137
1768/2137
1769/2137
1770/2137
1771/2137
1772/2137
1773/2137
1774/2137
1775/2137
1776/2137
1777/2137
1778/2137
1779/2137
1780/2137
1781/2137
1782/2137
1783/2137
1784/2137
1785/2137
1786/2137
1787/2137
1788/2137
1789/2137
1790/2137
1791/2137
1792/2137
1793/2137
1794/2137
1795/2137
1796/2137
1797/2137
1798/2137
1799/2137
1800/2137
1801/2137
1802/2137
1803/2137
1804/2137
1805/2137
1806/2137
1807/2137
1808/2137
1809/2137
1810/2137
1811/2137
1812/2137
1813/2137
1814/2137
1815/2137
1816/2137
1817/2137
1818/2137
1819/2137
1820/2137
1821/2137
1822/2137
1823/2137
1824/2137
1825/2137
1826/2137
1827/2137
1828/2137
1829/2137
1830/2137
1831/2137
1832/2137
1833/2137
1834/2137
1835/2137
1836/2137
1837/2137
1838/2137
1839/2137
1840/2137
1841/2137
1842/2137
1843/2137
1844/2137
1845/2137
1846/2137
1847/2137
1848/2137
1849/2137
1850/2137


493/2137
494/2137
495/2137
496/2137
497/2137
498/2137
499/2137
500/2137
501/2137
502/2137
503/2137
504/2137
505/2137
506/2137
507/2137
508/2137
509/2137
510/2137
511/2137
512/2137
513/2137
514/2137
515/2137
516/2137
517/2137
518/2137
519/2137
520/2137
521/2137
522/2137
523/2137
524/2137
525/2137
526/2137
527/2137
528/2137
529/2137
530/2137
531/2137
532/2137
533/2137
534/2137
535/2137
536/2137
537/2137
538/2137
539/2137
540/2137
541/2137
542/2137
543/2137
544/2137
545/2137
546/2137
547/2137
548/2137
549/2137
550/2137
551/2137
552/2137
553/2137
554/2137
555/2137
556/2137
557/2137
558/2137
559/2137
560/2137
561/2137
562/2137
563/2137
564/2137
565/2137
566/2137
567/2137
568/2137
569/2137
570/2137
571/2137
572/2137
573/2137
574/2137
575/2137
576/2137
577/2137
578/2137
579/2137
580/2137
581/2137
582/2137
583/2137
584/2137
585/2137
586/2137
587/2137
588/2137
589/2137
590/2137
591/2137
592/2137
593/2137
594/2137
595/2137
596/2137
597/2137
598/2137
599/2137
600/2137
601/2137
602/2137
603/2137
6

1363/2137
1364/2137
1365/2137
1366/2137
1367/2137
1368/2137
1369/2137
1370/2137
1371/2137
1372/2137
1373/2137
1374/2137
1375/2137
1376/2137
1377/2137
1378/2137
1379/2137
1380/2137
1381/2137
1382/2137
1383/2137
1384/2137
1385/2137
1386/2137
1387/2137
1388/2137
1389/2137
1390/2137
1391/2137
1392/2137
1393/2137
1394/2137
1395/2137
1396/2137
1397/2137
1398/2137
1399/2137
1400/2137
1401/2137
1402/2137
1403/2137
1404/2137
1405/2137
1406/2137
1407/2137
1408/2137
1409/2137
1410/2137
1411/2137
1412/2137
1413/2137
1414/2137
1415/2137
1416/2137
1417/2137
1418/2137
1419/2137
1420/2137
1421/2137
1422/2137
1423/2137
1424/2137
1425/2137
1426/2137
1427/2137
1428/2137
1429/2137
1430/2137
1431/2137
1432/2137
1433/2137
1434/2137
1435/2137
1436/2137
1437/2137
1438/2137
1439/2137
1440/2137
1441/2137
1442/2137
1443/2137
1444/2137
1445/2137
1446/2137
1447/2137
1448/2137
1449/2137
1450/2137
1451/2137
1452/2137
1453/2137
1454/2137
1455/2137
1456/2137
1457/2137
1458/2137
1459/2137
1460/2137
1461/2137
1462/2137


57/2137
58/2137
59/2137
60/2137
61/2137
62/2137
63/2137
64/2137
65/2137
66/2137
67/2137
68/2137
69/2137
70/2137
71/2137
72/2137
73/2137
74/2137
75/2137
76/2137
77/2137
78/2137
79/2137
80/2137
81/2137
82/2137
83/2137
84/2137
85/2137
86/2137
87/2137
88/2137
89/2137
90/2137
91/2137
92/2137
93/2137
94/2137
95/2137
96/2137
97/2137
98/2137
99/2137
100/2137
101/2137
102/2137
103/2137
104/2137
105/2137
106/2137
107/2137
108/2137
109/2137
110/2137
111/2137
112/2137
113/2137
114/2137
115/2137
116/2137
117/2137
118/2137
119/2137
120/2137
121/2137
122/2137
123/2137
124/2137
125/2137
126/2137
127/2137
128/2137
129/2137
130/2137
131/2137
132/2137
133/2137
134/2137
135/2137
136/2137
137/2137
138/2137
139/2137
140/2137
141/2137
142/2137
143/2137
144/2137
145/2137
146/2137
147/2137
148/2137
149/2137
150/2137
151/2137
152/2137
153/2137
154/2137
155/2137
156/2137
157/2137
158/2137
159/2137
160/2137
161/2137
162/2137
163/2137
164/2137
165/2137
166/2137
167/2137
168/2137
169/2137
170/2137
171/2137
172/2137

973/2137
974/2137
975/2137
976/2137
977/2137
978/2137
979/2137
980/2137
981/2137
982/2137
983/2137
984/2137
985/2137
986/2137
987/2137
988/2137
989/2137
990/2137
991/2137
992/2137
993/2137
994/2137
995/2137
996/2137
997/2137
998/2137
999/2137
1000/2137
1001/2137
1002/2137
1003/2137
1004/2137
1005/2137
1006/2137
1007/2137
1008/2137
1009/2137
1010/2137
1011/2137
1012/2137
1013/2137
1014/2137
1015/2137
1016/2137
1017/2137
1018/2137
1019/2137
1020/2137
1021/2137
1022/2137
1023/2137
1024/2137
1025/2137
1026/2137
1027/2137
1028/2137
1029/2137
1030/2137
1031/2137
1032/2137
1033/2137
1034/2137
1035/2137
1036/2137
1037/2137
1038/2137
1039/2137
1040/2137
1041/2137
1042/2137
1043/2137
1044/2137
1045/2137
1046/2137
1047/2137
1048/2137
1049/2137
1050/2137
1051/2137
1052/2137
1053/2137
1054/2137
1055/2137
1056/2137
1057/2137
1058/2137
1059/2137
1060/2137
1061/2137
1062/2137
1063/2137
1064/2137
1065/2137
1066/2137
1067/2137
1068/2137
1069/2137
1070/2137
1071/2137
1072/2137
1073/2137
1074/2137
1075/21

1795/2137
1796/2137
1797/2137
1798/2137
1799/2137
1800/2137
1801/2137
1802/2137
1803/2137
1804/2137
1805/2137
1806/2137
1807/2137
1808/2137
1809/2137
1810/2137
1811/2137
1812/2137
1813/2137
1814/2137
1815/2137
1816/2137
1817/2137
1818/2137
1819/2137
1820/2137
1821/2137
1822/2137
1823/2137
1824/2137
1825/2137
1826/2137
1827/2137
1828/2137
1829/2137
1830/2137
1831/2137
1832/2137
1833/2137
1834/2137
1835/2137
1836/2137
1837/2137
1838/2137
1839/2137
1840/2137
1841/2137
1842/2137
1843/2137
1844/2137
1845/2137
1846/2137
1847/2137
1848/2137
1849/2137
1850/2137
1851/2137
1852/2137
1853/2137
1854/2137
1855/2137
1856/2137
1857/2137
1858/2137
1859/2137
1860/2137
1861/2137
1862/2137
1863/2137
1864/2137
1865/2137
1866/2137
1867/2137
1868/2137
1869/2137
1870/2137
1871/2137
1872/2137
1873/2137
1874/2137
1875/2137
1876/2137
1877/2137
1878/2137
1879/2137
1880/2137
1881/2137
1882/2137
1883/2137
1884/2137
1885/2137
1886/2137
1887/2137
1888/2137
1889/2137
1890/2137
1891/2137
1892/2137
1893/2137
1894/2137
